In [13]:
import json 
import os 
import numpy as np
import math

In [14]:
def getJson(name):
    with open(name+'test_truth.json') as json_file:
        test_truth = json.load(json_file)
    with open(name+'test_rank.json') as json_file:
        test_rank = json.load(json_file)
    with open(name+'test_score.json') as json_file:
        test_score = json.load(json_file)
#         test_score = eval(test_score)
    with open(name+'test_item.json') as json_file:
        test_item = json.load(json_file)
        
#     print(type(test_truth), type(test_rank), type(test_score), type(test_item)) # all dict
#     print(len(test_truth), len(test_rank), len(test_score), len(test_item)) # all 150
    
    return test_truth,test_rank,test_score,test_item

In [15]:
def DCG(prec_list): #找出前n名的[1,1,1,0,...]
    dcg = 0
    for i in range(len(prec_list)):
        dcg += (2**prec_list[i]-1)/math.log2(i+2)
    return dcg

def NDCG(test_truth, test_rank, num_ndcg): #test_truth是每位User真正positive的item
    total_ndcg = 0

    for user in test_truth.keys():
#         print('user:', user)
        user_truth = test_truth[user]
        user_rank = test_rank[user]
#         print(len(user_truth), user_truth)
#         print(len(user_rank), user_rank)
        if len(user_truth) >= 32:
            print('Error!!!')
            break
        
        if not len(user_rank) == 32:
            print('Error!!!')
            break
        
        len_for_truth = len(user_truth)
        target = []
        for i in range(len(user_rank)):
            if i < len_for_truth:
                target.append(1)
            else:
                target.append(0)
#         print(sum(target), target)
        
        if not sum(target) == len_for_truth:
            print('Error!!!')
            break
        
        idcg = DCG(target[:num_ndcg])
        
        pre_list = []
        for item in user_rank[:num_ndcg]:
            if item in user_truth:
                pre_list.append(1)
            else:
                pre_list.append(0)
#         print('pre_list:', pre_list)
        if sum(pre_list) > len_for_truth:
            print('Error!!!')
            break
            
        dcg = DCG(pre_list)
        ndcg = dcg/idcg
        total_ndcg += ndcg
#         print('==================================================')
    avg_ndcg = total_ndcg/len(test_truth)
#     print('len(test_truth):', len(test_truth))
    return avg_ndcg

In [24]:
from sklearn.metrics import average_precision_score

def MAP(test_item, test_truth, test_score):
    Map_value = []
    for user in test_item.keys():
#         print('user:', user)
        user_item = test_item[user]
        user_truth = test_truth[user]
#         print('user_item:', user_item)
#         print('user_truth:', user_truth)
        if not len(user_item) == 32:
            print('Error!!!')
            break
        
        y_true = np.zeros(len(user_item))
        for tp in user_truth:
            y_true[user_item.index(tp)] = 1
#         print('y_true:', y_true)
        
        y_scores = test_score[user]
        Map_value.append(average_precision_score(y_true, y_scores))
    
#     print('len(test_truth):', len(Map_value),len(test_truth))
    return np.mean(Map_value)

In [36]:
def F1_score(prec,rec):
    f1 = 2*((prec*rec)/(prec+rec))
    return f1

def topN(test_rank, test_truth, n):
    print('Top', n)
    sumtarget = 0
    correct = 0
    for user in test_rank.keys():
#         print('user:', user)
        user_rank = test_rank[user]
        user_truth = test_truth[user]
        if not len(user_rank) == 32:
            print('Error!!!')
            break
            
        pos = np.zeros(len(user_rank))
        for tp in user_truth:
            pos[user_rank.index(tp)] = 1
#         print('pos:', pos)
#         print('sum(pos[:n]):',sum(pos[:n]))
        correct += sum(pos[:n])
        sumtarget += len(user_truth)
#         print('len(user_truth):', len(user_truth))

    print('correct:', correct) 
    print('sumtarget:', sumtarget)
    prec = correct/(len(test_rank)*n) #150*n
    recall = correct/sumtarget

    print('prec:', prec)
    print('recall:', recall)
    print('F1_score:', F1_score(prec, recall))
    print('*****')

In [37]:
#nameList = ['','ColdU','ColdY']
nameList = ['']#,'ColdY', 'ColdU']
num_of_y_test = 32
for name in nameList:
    test_truth,test_rank,test_score,test_item = getJson(name)
    print('Name:',name)
    print('NDCG@10:', NDCG(test_truth, test_rank, 10))
    print('MAP:', MAP(test_item, test_truth, test_score))
    topN(test_rank, test_truth, 1)
    topN(test_rank, test_truth, 5)
    print('==================================================')

Name: 
NDCG@10: 0.5291227070395198
MAP: 0.44294631023012415
Top 1
correct: 77.0
sumtarget: 1078
prec: 0.5133333333333333
recall: 0.07142857142857142
F1_score: 0.1254071661237785
*****
Top 5
correct: 335.0
sumtarget: 1078
prec: 0.44666666666666666
recall: 0.310760667903525
F1_score: 0.3665207877461707
*****
